In [3]:
from pymilvus import MilvusClient
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os
import pickle
import tqdm
import pandas as pd
import requests
import torch
import numpy as np

#########   KEY IN YOUR IP  #########
client = MilvusClient(
    uri="http://192.168.1.111:19530"
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
model.to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
def get_clip_embedding(image_path):
    image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt")
    inputs = inputs.to(device)
    outputs = model.get_image_features(**inputs)
    outputs = outputs.cpu()  
    return outputs

In [ ]:
image_path = input('Paste the path of your image here:')
image_embedding = get_clip_embedding(image_path)
image_embedding = image_embedding.detach().numpy()
image_embedding = image_embedding.squeeze()
image_embedding = np.array(image_embedding)

In [ ]:
data = [image_embedding]
res = client.search(
    collection_name="image", # Replace with the actual name of your collection
    data=data,
    limit=1,  # Max. number of search results to return
    search_params={"metric_type": "COSINE", "params": {}}, # Search parameters
    partition_names=["_default"] # Partition names to search in
)

id = int(input('Key in id:'))
xxx = input('Key in name:')

partition_id = res[0][0]['id']
print(partition_id)
data = {"id": id, "vector": image_embedding, "name": xxx}
res = client.insert(
    collection_name="image",
    data=data,
    partition_name="partition"+str(partition_id)
)